In [46]:
import json
import os
import pickle
import sys
import subprocess
sys.path.append("../..")
from src.training.dataset_utils import read_conll_file, examples_to_indices
from src.training.train_utils import evaluate_ner, form_ner_pred_matrix


from sklearn.metrics import classification_report
import tensorflow as tf

In [47]:
base_path = "/Users/Carol/Google Drive/"
dev_file = os.path.join(base_path, "nlp_data/recipe_data/20200523_food_gold_dev.conll")
model_file = "/Users/Carol/Google Drive/nlp_data/output/20200523_22_06_28/20200523_22_06_28_food_ner_epoch_5_dev_f1_0.9851520816163143.h5"
mappings_file = "/Users/Carol/Google Drive/nlp_data/output/20200523_22_06_28/20200523_22_06_28_food_ner_mappings.pkl"

In [48]:
model = tf.keras.models.load_model(model_file)

In [49]:
mappings = pickle.load(open(mappings_file, "rb"))
label_to_index = mappings['label_to_index']
token_to_index = mappings['token_to_index']
index_to_label = {v:k for k,v in label_to_index.items()}

In [50]:
dev_dataset = read_conll_file(dev_file)
dev_sentences = examples_to_indices(dev_dataset, label_to_index, token_to_index)

In [51]:
label_mappings = list(index_to_label.items())
label_mappings.sort()
label_strings = [x[1] for x in label_mappings]
y_pred = []
y_true = []
for sent in dev_sentences:
    preds = model.predict_on_batch(form_ner_pred_matrix(sent['tokens']))
    y_pred.extend(np.argmax(preds, axis=-1)[0])
    y_true.extend(sent['labels'])
metrics = classification_report(y_true, y_pred, target_names = label_strings,
                                output_dict=False)


In [52]:
# token-level metrics
print(metrics)

              precision    recall  f1-score   support

      I-FOOD       0.93      0.86      0.89       278
           O       0.99      0.99      0.99      9464
      B-FOOD       0.95      0.94      0.95      1175

    accuracy                           0.99     10917
   macro avg       0.96      0.93      0.94     10917
weighted avg       0.99      0.99      0.99     10917



In [55]:
# write to conll format for use of conll perl script to calculate entity-level metrics
outfile = "/Users/Carol/Google Drive/nlp_data/output/20200523_22_06_28/dev_conll.txt"
with open(outfile, "w") as out:
    ctr = 0
    for doc in dev_sentences:
        for token in doc['raw_tokens']:
            out.write(f"{token} {index_to_label[y_true[ctr]]} {index_to_label[y_pred[ctr]]}\n")
            ctr += 1
        out.write("\n")
            

In [56]:
infile = "/Users/Carol/Google Drive/nlp_data/output/20200523_22_06_28/dev_conll.txt"
outfile = '/Users/Carol/Google Drive/nlp_data/output/20200523_22_06_28/eval.txt'
conlleval_script_path = "/Users/Carol/Dropbox/repos/food/src/evaluation"
os.chdir(conlleval_script_path)
cmd = "perl conlleval.pl < {} > {}".format(json.dumps(infile), json.dumps(outfile))
os.system(cmd)

0